In [14]:
### import needed libraries

import os
import datetime as dt
import pandas as pd
import openpyxl 
import numpy as np
pd.set_option('display.max_columns', None)

In [15]:
###Load Data 

df = pd. read_excel (r'./online_retail_II.xlsx', sheet_name='Year 2009-2010' , engine='openpyxl')
df.head(2)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom


In [16]:
### display dat 
print(df.columns)
df.head(2)

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country'],
      dtype='object')


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom


In [17]:
### create a new filtered data set for UK

df1 = df[df['Country'] == 'United Kingdom']
df1.head(2)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom


In [18]:
### review data 
df1.describe()

,Quantity,Price,Customer ID
count,485852.000000,485852.000000,379423.000000
mean,9.116039,4.543470,15559.935694
std,85.883463,149.623198,1593.744626
min,-9600.000000,-53594.360000,12346.000000
25%,1.000000,1.250000,14210.000000
50%,3.000000,2.100000,15581.000000
75%,10.000000,4.210000,16938.000000
max,10200.000000,25111.090000,18287.000000


In [19]:
df1.Country.value_counts()

United Kingdom    485852
Name: Country, dtype: int64

In [20]:
### Calcualte RFM 

# define today parameter for recently calculation
today_date = dt.datetime(2010, 12, 11)

RFM = df1.groupby('Customer ID').agg({'InvoiceDate': lambda date: (today_date - date.max()).days,
                                     'Invoice': lambda num: len(num),
                                     'Price': lambda Price: Price.sum()})

In [21]:
### change columns labels. 

RFM.columns = ['Recency', 'Frequency', 'Monetary']

In [22]:
### display RFM dataframe 
RFM

,Recency,Frequency,Monetary
Customer ID,,,
12346.0,67,46,578.36
12608.0,40,16,26.04
12745.0,122,22,67.98
12746.0,163,20,73.95
12747.0,5,162,716.11
...,...,...,...
18283.0,18,230,498.82
18284.0,65,29,116.09
18285.0,296,12,100.20


In [23]:
### Recency groups according to score
RFM["RecencyScore"] = pd.qcut(RFM['Recency'], 5, labels=[5, 4, 3, 2, 1])

###Frequency groups according to score
RFM["FrequencyScore"] = pd.qcut(RFM['Frequency'], 5, labels=[1, 2, 3, 4, 5])


###Monetary groups according to score
RFM["MonetaryScore"] = pd.qcut(RFM['Monetary'], 5, labels=[1, 2, 3, 4, 5])

###Creating an overall score. 
RFM["RFM_SCORE"] = (RFM['RecencyScore'].astype(str) +
                    RFM['FrequencyScore'].astype(str) +
                    RFM['MonetaryScore'].astype(str))


In [56]:
### Display value counts 

pd.DataFrame([RFM['RecencyScore'].value_counts(),RFM['FrequencyScore'].value_counts(), RFM['MonetaryScore'].value_counts()]).T

,RecencyScore,FrequencyScore,MonetaryScore
1,807,837,807
2,805,789,807
3,805,804,807
4,735,806,807
5,883,799,807


In [57]:
# RFM segmentation mapping
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At_Risk',
    r'[1-2]5': 'Cant_Loose',
    r'3[1-2]': 'About_to_Sleep',
    r'33': 'Need_Attention',
    r'[3-4][4-5]': 'Loyal_Customers',
    r'41': 'Promising',
    r'51': 'New_Customers',
    r'[4-5][2-3]': 'Potential_Loyalists',
    r'5[4-5]': 'Champions'
}

In [59]:
# add segments column
RFM['Segment'] = RFM['RecencyScore'].astype(str) + RFM['FrequencyScore'].astype(str)
RFM.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
Customer ID,,,,,,,,
12346.0,67,46,578.36,3,3,5,335,33
12608.0,40,16,26.04,3,2,1,321,32
12745.0,122,22,67.98,2,2,2,222,22
12746.0,163,20,73.95,2,2,2,222,22
12747.0,5,162,716.11,5,5,5,555,55


In [61]:
RFM['Segment'] = RFM['Segment'].replace(seg_map, regex=True)


In [66]:
RFM[["Segment", "Recency", "Frequency", "Monetary"]].groupby("Segment").agg(["mean", "count","std"])

Recency                    Frequency        \
                           mean count        std        mean count   
Segment                                                              
About_to_Sleep        52.471875   320  10.191850   15.459375   320   
At_Risk              161.421053   570  80.158553   58.215789   570   
Cant_Loose           125.534884    86  50.772780  232.046512    86   
Champions              6.732899   614   4.512118  261.314332   614   
Hibernating          209.668410   956  90.729319   13.424686   956   
Loyal_Customers       36.721358   707  15.875762  173.205092   707   
Need_Attention        50.276316   152  10.264191   45.730263   152   
New_Customers          7.783333    60   4.333909    7.216667    60   
Potential_Loyalists   17.157895   475   9.308945   36.709474   475   
Promising             24.421053    95   5.158300    8.200000    95   

                                   Monetary                     
                            std        mean count          std  
Segment                                                         
About_to_Sleep         8.642947  157.263844   320  1002.325505  
At_Risk               24.071308  185.023512   570   110.579890  
Cant_Loose           162.221048  705.643500    86   686.948763  
Champions            370.497612  851.894502   614  1413.504487  
Hibernating            8.644677  157.522742   956  1387.520343  
Loyal_Customers      154.443657  520.840983   707   442.555837  
Need_Attention         8.535838  411.011908   152  3227.238742  
New_Customers          3.987658   33.589500    60    43.270695  
Potential_Loyalists   13.253445  126.599411   475   118.514052  
Promising              3.891070   33.052632    95    32.215465